In [1]:
import agentpy as ap
import numpy as np
import matplotlib.pyplot as plt
import math
import IPython
import json
import random

In [2]:
myJson={
    "carsAmount": 0,
    "steps":[]
}

def convertToJson():
    with open("variables.json", "w") as outfile:
        json.dump(myJson, outfile)

In [3]:
class Semaphore(ap.Agent):
    def setup(self):
        self.step_time = 0.1
        self.direction = [0, 1]
        self.state = 0 # green = 0, yellow = 1, red = 2
        self.state_time = 0

        self.state_time_total = 0

        self.green_duration = 50
        self.yellow_duration = 5
        self.red_duration = 45

#     def update(self) :
#         self.state_time += self.step_time

#         if self.state == 0:
#             print("VERDE")
#             self.state_time_total += self.step_time
#             if self.state_time >= self.green_duration:
#                 self.state = 1
#                 self.state_time = 0

#         if self.state == 1:
#             if self.state_time >= self.yellow_duration:
#                 self.state = 2
#                 self.state_time = 0

#         if self.state == 2:
#             if self.state_time >= self.red_duration:
#                 self.state = 0
#                 self.state_time = 0

                
#         return False

    def set_green(self):
        self.state = 0
        self.state_time = 0
    
    def set_yellow(self):
        self.state = 1
        self.state_time = 0

    def set_red(self):
        self.state = 2
        self.state_time = 0
    
    def cars(self):
        count = 0
        """ Este método calcula el número de autos que están frente al semáforo a m enos de 150 metros."""

        # Obten la posición actual del semáforo
        p = self.model.avenue.positions[self]
        for car in self.model.cars:
            if car != self:
                dot_p1 = self.direction[0] * car.direction[0] + self.direction[1] * car.direction[1]

                p2 = self.model.avenue.positions[car]
                dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]

                if dot_p1 < 0 and dot_p2 < 0:
                    d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)

                        co
                    if d < 150:unt += 1
        
        return count

In [4]:
class Car(ap.Agent):
    def setup(self):
        self.step_time = 0.1
        self.direction = [0, 1]
        self.speed = 0.0
        self.max_speed = 16.66
        self.state = 1 # 0 = chocado, 1 = ok

    def update_position(self):
        if self.state == 0:
            return
        
        # Posibilidad de dar vuelta
        if self.model.avenue.positions[self][1] > self.p.size//2-20 and self.model.avenue.positions[self][1] < self.p.size//2-5 and self.direction == [0,1]:
            if random.randint(1, 6) == 1:
                self.direction = [1,0]
                self.model.avenue.move_to(self, [self.model.avenue.positions[self][0], self.p.size//2-20])
        elif self.model.avenue.positions[self][1] > self.p.size//2 and self.model.avenue.positions[self][1] < self.p.size//2+5 and self.direction == [0,-1]:
            if random.randint(1, 6) == 1:
                self.direction = [-1,0]
                self.model.avenue.move_to(self, [self.model.avenue.positions[self][0], self.p.size//2])
        elif self.model.avenue.positions[self][0] > self.p.size//2-20 and self.model.avenue.positions[self][0] < self.p.size//2-5 and self.direction == [1,0]:
            if random.randint(1, 6) == 1:
                self.direction = [0,-1]
                self.model.avenue.move_to(self, [self.p.size//2-20, self.model.avenue.positions[self][1]])
        elif self.model.avenue.positions[self][0] > self.p.size//2 and self.model.avenue.positions[self][0] < self.p.size//2+5 and self.direction == [-1,0]:
            if random.randint(1, 6) == 1:
                self.direction = [0,1]
                self.model.avenue.move_to(self, [self.p.size//2, self.model.avenue.positions[self][1]])

        
        self.model.avenue.move_by(self, [self.speed*self.step_time*self.direction[0], self.speed*self.step_time*self.direction[1]])

    def update_speed(self):
        if self.state == 0:
            return
        
        p = self.model.avenue.positions[self]

        min_car_distance = 10000000
        for car in self.model.cars:
            if car == self:
                continue

            dot_p1 = self.direction[0]*car.direction[0] + self.direction[1]*car.direction[1]

            p2 = self.model.avenue.positions[car]
            dot_p2 = (p2[0] - p[0])*self.direction[0] + (p2[1] - p[1])*self.direction[1]

            if dot_p1 > 0 and dot_p2 > 0:
                d = math.sqrt((p2[0] - p[0])**2 + (p2[1] - p[1])**2)
                if d < min_car_distance:
                    min_car_distance = d

        min_semaphore_distance = 10000000
        semaphore_state = 0
        for semaphore in self.model.semaphores:

            dot_p1 = self.direction[0]*semaphore.direction[0] + self.direction[1]*semaphore.direction[1]

            p2 = self.model.avenue.positions[semaphore]
            dot_p2 = (p2[0] - p[0])*self.direction[0] + (p2[1] - p[1])*self.direction[1]

            if dot_p1 < 0 and dot_p2 > 0:
                d = math.sqrt((p2[0] - p[0])**2 + (p2[1] - p[1])**2)
                if d < min_semaphore_distance:
                    min_semaphore_distance = d
                    semaphore_state = semaphore.state
                
        if min_car_distance < 2:
            self.speed = 0
            self.state = 1

        elif min_car_distance < 10:
            self.speed = np.maximum(self.speed - 50*self.step_time, 0)
        
        elif min_car_distance < 15:
            self.speed = np.maximum(self.speed - 50*self.step_time, 0)
        
        elif min_semaphore_distance < 20 and semaphore_state == 1:
            self.speed = np.minimum(self.speed + 2.5*self.step_time, self.max_speed)

        elif min_semaphore_distance < 40 and semaphore_state == 1:
            self.speed = np.maximum(self.speed - 2.5*self.step_time, 5)

        elif min_semaphore_distance < 30 and semaphore_state == 2:
            self.speed = np.maximum(self.speed - 10*self.step_time, 0)

        elif min_semaphore_distance < 50 and semaphore_state == 2:
            self.speed = np.maximum(self.speed - 2.5*self.step_time, 5)

        else:
            self.speed = np.minimum(self.speed + 2.0*self.step_time, self.max_speed)

In [5]:
stepCount=0
class AvenueModel(ap.Model):
    def setup(self):

        myJson["carsAmount"] = self.p.cars

        self.cars = ap.AgentList(self, self.p.cars, Car)
        self.cars.step_time = self.p.step_time
        self.cars.speed = self.p.v0
        self.stepCount=0

        # c_north = self.p.cars//4
        # c_south = self.p.cars//4
        # c_east = self.p.cars//4
        # c_west = self.p.cars - c_north - c_south - c_east

        c_north = 2
        c_south = 4
        c_east = 6
        c_west = 8

        for k in range(c_north):
            self.cars[k].direction = [0, 1]

        for k in range(c_south):
            self.cars[c_north + k].direction = [0, -1]
        
        # Direccion a carros este y oeste

        for k in range(c_east):
            self.cars[c_north + c_south + k].direction = [1, 0]
        
        for k in range(c_west):
            self.cars[c_north + c_south + c_east + k].direction = [-1, 0]
        
        self.semaphores = ap.AgentList(self, 4, Semaphore)
        self.semaphores.step_time = self.p.step_time
        self.semaphores.green_duration = self.p.green
        self.semaphores.yellow_duration = self.p.yellow
        self.semaphores.red_duration = self.p.red
        self.semaphores[0].direction = [0, 1]
        self.semaphores[1].direction = [0, -1]
        self.semaphores[2].direction = [1, 0]
        self.semaphores[3].direction = [-1, 0]

        self.avenue = ap.Space(self, shape = [self.p.size, self.p.size], torus = True)

        self.avenue.add_agents(self.semaphores, random=True)
        self.avenue.move_to(self.semaphores[0], [self.p.size//2-30, self.p.size//2+60])
        self.avenue.move_to(self.semaphores[1], [self.p.size//2+10, self.p.size//2-60])


        self.avenue.move_to(self.semaphores[2], [self.p.size//2+60, self.p.size//2+10])
        self.avenue.move_to(self.semaphores[3], [self.p.size//2-60, self.p.size//2-30])
        self.semaphores[2].state = 2
        self.semaphores[3].state = 2

        self.avenue.add_agents(self.cars, random=True)

        # Acomodar carros
        for k in range(c_north):
            self.avenue.move_to(self.cars[k], [self.p.size//2, 10*(k+1)])

        for k in range(c_south):
            self.avenue.move_to(self.cars[k+c_north], [self.p.size//2-20, self.p.size - 10*(k+1)])

        for k in range(c_east):
            self.avenue.move_to(self.cars[k+c_north+c_south], [10*(k+1), self.p.size//2-20])
        
        for k in range(c_west):
            self.avenue.move_to(self.cars[k+c_north+c_south+c_east], [self.p.size - 10*(k+1), self.p.size//2])
        
        #Usamos la info para generar el JSON
        cars=[]
        semaphores=[]
        
        for c in range(len(self.cars)):
            currentInfo={
                "id":self.cars[c].id,
                "x":self.model.avenue.positions[self.cars[c]][0],
                "z":self.model.avenue.positions[self.cars[c]][1],
                "y":self.p.y,
                "direction":self.cars[c].direction
            }
            cars.append(currentInfo)
        
        for s in range(len(self.semaphores)):
            currentInfo={
                "x":self.model.avenue.positions[self.semaphores[s]][0],
                "z":self.model.avenue.positions[self.semaphores[s]][1],
                "y":self.p.y,
                "direction":self.semaphores[s].direction,
                "state":self.semaphores[s].state,
                
            }
            semaphores.append(currentInfo)
        
        myJson["steps"].append({
            "id":self.stepCount,
            "cars":cars,
            "semaphores":semaphores
        })
 
    def step(self):
        self.stepCount+=1
        
        # Definir variables para controlar semaforos
        self.step_time = 0.1
        self.state_time = 0

        self.cars.update_position()
        self.cars.update_speed()
        
        #Usamos la info para generar el JSON
        cars=[]
        semaphores=[]
        for c in range(len(self.cars)):
            currentInfo={
                "id":self.cars[c].id,
                "x":self.model.avenue.positions[self.cars[c]][0],
                "z":self.model.avenue.positions[self.cars[c]][1],
                "y":self.p.y,
                "direction":self.cars[c].direction
            }
            cars.append(currentInfo)
        
        for s in range(len(self.semaphores)):
            currentInfo={
                "x":self.model.avenue.positions[self.semaphores[s]][0],
                "z":self.model.avenue.positions[self.semaphores[s]][1],
                "y":self.p.y,
                "direction":self.semaphores[s].direction,
                "state":self.semaphores[s].state,
                
            }
            semaphores.append(currentInfo)
        
        myJson["steps"].append({
            "id":self.stepCount,
            "cars":cars,
            "semaphores":semaphores
        })
        
    def updateSemaphores(self):
        self.state_time += self.step_time
        
        if self.state_time == self.p.green:
            for i in range(len(self.semaphores)):
                if self.semaphores[i].state == 0:
                    self.semaphores[i].set_yellow()
        
        if self.state_time == self.p.red:
            self.vote()

        
    
    def vote(self):
        # Sistema de votacion
        # Prender el semaforo con mayor cantidad de coches
        # Prender dos semaforos por votacion (Semaforos paralelos)

        # Cuando el semaforo verde cambie a rojo, se realiza la votacion
        # Se revisa cual semaforo tiene mas carros
        # Se pone en verde el semaforo con mas carros detras junto con su paralelo
        # Se le suma tiempo por cada carro detras

        # Si existe un empate

        maxCars = 0
        semaphore_i = 0
        semaphore_pair_i = 0

        semaphore_loser_i = 0

        for i in range(len(self.semaphores)):
            if self.semaphores[i].cars() > maxCars:
                semaphore_i = i
                maxCars = self.semaphores[i].cars()

        # Cambiar todos los semaforos

        if semaphore_i == 0:
            semaphore_pair_i = 1
            semaphore_loser_i = 2
        elif semaphore_i == 1:
            semaphore_pair_i = 0
            semaphore_loser_i = 2
        elif semaphore_i == 2:
            semaphore_pair_i = 3
            semaphore_loser_i = 0
        elif semaphore_i == 3:
            semaphore_pair_i = 2
            semaphore_loser_i = 0


        # Cambiar semaforos elegidos a verde
        self.semaphores[semaphore_i].set_green()
        self.semaphores[semaphore_pair_i].set_green()

        # Cambiar semaforos perdedores a rojo
        self.semaphores[semaphore_loser_i].set_yellow()
        self.semaphores[semaphore_loser_i+1].set_yellow()


        return

    def update(self):
        crashes = len(self.cars.select(self.cars.state == 0))
        self.record('Colisiones', crashes)
    
    def end(self):
        crashes = len(self.cars.select(self.cars.state == 0))
        self.report('Colisiones', crashes)
        convertToJson()

In [6]:
parameters = {
    'steps': 100,
    'step_time': 0.1,
    'size': 500,
    'green': 15,
    'yellow': 5,
    'red': 20,
    'cars': 20,
    'v0': 0,
    'y':0
}

In [7]:
def animation_plot_single(m, ax):
    ax.set_title(f"Avenida t={m.t*m.p.step_time:.2f}")

    colors = ["green", "yellow", "red"]

    pos_s1 = m.avenue.positions[m.semaphores[0]]
    ax.scatter(*pos_s1, s=20, c=colors[m.semaphores[0].state])

    pos_s2 = m.avenue.positions[m.semaphores[1]]
    ax.scatter(*pos_s2, s=20, c=colors[m.semaphores[1].state])
    
    pos_s3 = m.avenue.positions[m.semaphores[2]]
    ax.scatter(*pos_s3, s=20, c=colors[m.semaphores[2].state])
    
    pos_s4 = m.avenue.positions[m.semaphores[3]]
    ax.scatter(*pos_s4, s=20, c=colors[m.semaphores[3].state])

    ax.set_xlim(0, m.avenue.shape[0])
    ax.set_ylim(0, m.avenue.shape[1])
    
    count = 0

    for car in m.cars:
        pos_c = m.avenue.positions[car]
        if count < m.p.cars//2:
            ax.scatter(*pos_c, s=20, c="black")
        else:
            ax.scatter(*pos_c, s=20, c="blue")
        count += 1

    ax.set_axis_off()
    ax.set_aspect('equal', 'box')

def animation_plot(m, p):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=20))

In [8]:
model = AvenueModel(parameters)
results = model.run()

state:  0
 time:  0.1
VERDE
state:  0
 time:  0.1
VERDE
state:  2
 time:  0.1
ROJO
state:  2
 time:  0.1
ROJO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
Completed: 1 stepsstate:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
Completed: 2 stepsstate:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
Completed: 3 stepsstate:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
Completed: 4 stepsstate:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
Complet

In [9]:
# Autos colisionados en cada iteración
results.variables.AvenueModel

,Colisiones
t,
0,0
1,0
2,0
3,0
4,0
...,...
96,0
97,0
98,0


In [10]:
# Información al final de la simulación
results.reporters

,seed,Colisiones
0,291929880976175596475524069402781644963,0


In [11]:
animation_plot(AvenueModel, parameters)

state:  0
 time:  0.1
VERDE
state:  0
 time:  0.1
VERDE
state:  2
 time:  0.1
ROJO
state:  2
 time:  0.1
ROJO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time: 

state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
sta

state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
trigger:  ['caca seca', 'caca seca', 'caca seca', 'caca seca']
state:  1
 time:  0.1
AMARILLO
state:  1
 time:  0.1
AMARILLO
sta